In [1]:
import pandas as pd
from google.cloud import storage
# Importazione necessaria per il parametro quoting
import csv 

# --- CONFIGURAZIONE (Immagino che questi siano corretti) ---
BUCKET_NAME = "data-science-project-luca-october-2025" 
GCS_FILE_PATH = "styles.csv"       
LOCAL_DOWNLOAD_PATH = "styles_local.csv" 

# ------------------------------------------------------------

# 1. DOWNLOAD (come prima)
try:
    print("Tentativo di download...")
    storage_client = storage.Client()
    bucket = storage_client.bucket(BUCKET_NAME)
    blob = bucket.blob(GCS_FILE_PATH)
    blob.download_to_filename(LOCAL_DOWNLOAD_PATH)
    print("SUCCESS: File scaricato.")
    
except Exception as e:
    print(f"ERRORE di Download GCS: {e}")


# 2. CARICAMENTO PANDAS CON GESTIONE ERRORI
try:
    df = pd.read_csv(
        LOCAL_DOWNLOAD_PATH, 
        sep=',',               
        engine='python',       
        encoding="ISO-8859-1",
        # PARAMETRO RISOLUTIVO: Forza Pandas a ignorare i problemi di citazione
        quoting=csv.QUOTE_NONE 
    )
    
    print("\nSUCCESS: Caricamento riuscito con forzatura CSV.")
    print(f"Righe: {len(df)}, Colonne: {len(df.columns)}")
    print("Intestazioni delle Colonne:")
    print(df.columns.tolist())
    print("\nPrime 5 righe del DataFrame:")
    print(df.head())
    
except Exception as e:
    print(f"\nERRORE FINALE: {e}. Il file è irrimediabilmente malformato.")

Tentativo di download...
SUCCESS: File scaricato.

ERRORE FINALE: Expected 10 fields in line 6044, saw 11. Il file è irrimediabilmente malformato.


In [3]:
import pandas as pd
import csv

# ... (Omesso codice di download da GCS, che ha funzionato) ...
LOCAL_DOWNLOAD_PATH = "styles_local.csv"

# CARICAMENTO PANDAS CON GESTIONE DEGLI ERRORI AGGIORNATA
try:
    print("\nTentativo di caricamento ignorando le righe malformate (on_bad_lines='skip')...")
    
    df = pd.read_csv(
        LOCAL_DOWNLOAD_PATH, 
        sep=',',               
        engine='python',       
        encoding="ISO-8859-1",
        quoting=csv.QUOTE_NONE,
        # NUOVO PARAMETRO CHIAVE: ignora le righe che danno errore
        on_bad_lines='skip' 
    )
    
    print("\n✅ SUCCESS: DataFrame caricato, righe problematiche saltate.")
    print(f"Righe finali nel DataFrame: {len(df)}") 
    print(f"Intestazioni delle Colonne: {df.columns.tolist()}")
    print("\nPrime 5 righe (finalmente!):")
    print(df.head())
    
except Exception as e:
    print(f"\nERRORE CRITICO: Fallimento totale del caricamento. Passare al cambio di dataset. Errore: {e}")


Tentativo di caricamento ignorando le righe malformate (on_bad_lines='skip')...

✅ SUCCESS: DataFrame caricato, righe problematiche saltate.
Righe finali nel DataFrame: 44424
Intestazioni delle Colonne: ['id', 'gender', 'masterCategory', 'subCategory', 'articleType', 'baseColour', 'season', 'year', 'usage', 'productDisplayName']

Prime 5 righe (finalmente!):
      id gender masterCategory subCategory  articleType baseColour  season  \
0  15970    Men        Apparel     Topwear       Shirts  Navy Blue    Fall   
1  39386    Men        Apparel  Bottomwear        Jeans       Blue  Summer   
2  59263  Women    Accessories     Watches      Watches     Silver  Winter   
3  21379    Men        Apparel  Bottomwear  Track Pants      Black    Fall   
4  53759    Men        Apparel     Topwear      Tshirts       Grey  Summer   

     year   usage                             productDisplayName  
0  2011.0  Casual               Turtle Check Men Navy Blue Shirt  
1  2012.0  Casual             Peter

In [4]:
# Panoramica rapida dei tipi di dati e dei valori mancanti
print("--- Riepilogo del DataFrame ---")
df.info()

# Controlla la distribuzione dei valori mancanti in percentuale
print("\n--- Percentuale di Valori Mancanti per Colonna ---")
missing_percentage = (df.isnull().sum() / len(df)) * 100
print(missing_percentage[missing_percentage > 0].sort_values(ascending=False))

--- Riepilogo del DataFrame ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44424 entries, 0 to 44423
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  44424 non-null  int64  
 1   gender              44424 non-null  object 
 2   masterCategory      44424 non-null  object 
 3   subCategory         44424 non-null  object 
 4   articleType         44424 non-null  object 
 5   baseColour          44409 non-null  object 
 6   season              44403 non-null  object 
 7   year                44423 non-null  float64
 8   usage               44107 non-null  object 
 9   productDisplayName  44417 non-null  object 
dtypes: float64(1), int64(1), object(8)
memory usage: 3.4+ MB

--- Percentuale di Valori Mancanti per Colonna ---
usage                 0.713578
season                0.047272
baseColour            0.033766
productDisplayName    0.015757
year                  0.002251
dt

In [5]:
# Controlla la distribuzione delle categorie principali
print("\n--- Distribuzione della Variabile Target: masterCategory ---")
print(df['masterCategory'].value_counts())


--- Distribuzione della Variabile Target: masterCategory ---
masterCategory
Apparel           21397
Accessories       11274
Footwear           9219
Personal Care      2403
Free Items          105
Sporting Goods       25
Home                  1
Name: count, dtype: int64


In [6]:
# Rimuovi le categorie con meno di 100 istanze per evitare rumore e bias eccessivi
MIN_COUNT = 100 
categories_to_keep = df['masterCategory'].value_counts()
categories_to_keep = categories_to_keep[categories_to_keep >= MIN_COUNT].index

df_ml = df[df['masterCategory'].isin(categories_to_keep)]

print(f"Righe dopo aver rimosso le categorie con meno di 100 istanze: {len(df_ml)}")

# Aggiorna la distribuzione del target
print("\n--- Nuova Distribuzione del Target ---")
print(df_ml['masterCategory'].value_counts())

Righe dopo aver rimosso le categorie con meno di 100 istanze: 44398

--- Nuova Distribuzione del Target ---
masterCategory
Apparel          21397
Accessories      11274
Footwear          9219
Personal Care     2403
Free Items         105
Name: count, dtype: int64


In [7]:
# Seleziona le colonne categoriali per l'encoding
categorical_cols = ['gender', 'subCategory', 'articleType', 'baseColour', 'season', 'usage']

# Applica One-Hot Encoding
df_encoded = pd.get_dummies(df_ml, columns=categorical_cols, drop_first=True)

print(f"\nNumero di colonne dopo l'encoding (molte di più!): {len(df_encoded.columns)}")
print("Le prime nuove colonne:", df_encoded.columns.tolist()[:15])


Numero di colonne dopo l'encoding (molte di più!): 242
Le prime nuove colonne: ['id', 'masterCategory', 'year', 'productDisplayName', 'gender_Girls', 'gender_Men', 'gender_Unisex', 'gender_Women', 'subCategory_Apparel Set', 'subCategory_Bags', 'subCategory_Bath and Body', 'subCategory_Beauty Accessories', 'subCategory_Belts', 'subCategory_Bottomwear', 'subCategory_Cufflinks']


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

# Riempi i NaN in productDisplayName per evitare errori
df_encoded['productDisplayName'] = df_encoded['productDisplayName'].fillna('')

# Inizializza il vettorizzatore TF-IDF
tfidf = TfidfVectorizer(stop_words='english', max_features=500) # Limita a 500 parole più comuni/importanti

# Adatta e trasforma i nomi dei prodotti
tfidf_matrix = tfidf.fit_transform(df_encoded['productDisplayName'])

# Converti la matrice TF-IDF in un DataFrame per unirla al DataFrame principale
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf.get_feature_names_out(), index=df_encoded.index)

# Unisci le feature TF-IDF al DataFrame codificato
df_final = pd.concat([df_encoded.drop(columns=['productDisplayName', 'id']), tfidf_df], axis=1)

print(f"\nNumero finale di colonne/feature per l'ML: {len(df_final.columns)}")


Numero finale di colonne/feature per l'ML: 740


In [10]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# 1. Preparazione della Variabile Target (Y)
# Il modello ML vuole numeri, non stringhe. Usiamo LabelEncoder.

# Rimuovi la colonna masterCategory dal DataFrame finale prima di codificarla
Y_text = df_final['masterCategory']

# Codifica Y_text (es. 'Apparel' -> 0, 'Accessories' -> 1)
le = LabelEncoder()
Y = le.fit_transform(Y_text) 

# 2. Preparazione delle Feature (X)
# Tutte le colonne rimanenti sono le nostre feature numeriche (incluse quelle One-Hot e TF-IDF)
X = df_final.drop(columns=['masterCategory']) 


# 3. Suddivisione Training/Testing (80% Training, 20% Testing)
X_train, X_test, Y_train, Y_test = train_test_split(
    X, 
    Y, 
    test_size=0.2, 
    random_state=42,
    stratify=Y # Mantiene le proporzioni dello sbilanciamento del target in entrambi i set
)

print(f"Dimensione Training Set (X_train): {X_train.shape}")
print(f"Dimensione Testing Set (X_test): {X_test.shape}")

Dimensione Training Set (X_train): (35518, 739)
Dimensione Testing Set (X_test): (8880, 739)


In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import numpy as np

# Inizializza il modello
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, class_weight='balanced')

print("\nInizio addestramento del Random Forest...")
# Addestra il modello sul set di Training
rf_model.fit(X_train, Y_train)
print("Addestramento completato.")

# Previsione sul set di Testing
Y_pred = rf_model.predict(X_test)


Inizio addestramento del Random Forest...
Addestramento completato.


In [12]:
# 1. Calcola l'Accuracy
accuracy = accuracy_score(Y_test, Y_pred)
print(f"\nAccuracy Generale del Modello: {accuracy:.4f}")


# 2. Report Completo di Classificazione
# Riconverti i codici numerici del target nei nomi delle categorie per un report leggibile
target_names = le.classes_ 

print("\n--- Report di Classificazione per Categoria ---")
print(classification_report(Y_test, Y_pred, target_names=target_names))

# 3. Matrice di Confusione (visualizzazione grafica)
# Per il portfolio è essenziale! 
cm = confusion_matrix(Y_test, Y_pred)

# (Codice per visualizzare la matrice con Matplotlib/Seaborn da aggiungere al Notebook)
# Ad esempio:
# import seaborn as sns
# import matplotlib.pyplot as plt
# plt.figure(figsize=(10, 7))
# sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=target_names, yticklabels=target_names)
# plt.xlabel('Predicted')
# plt.ylabel('True')
# plt.title('Confusion Matrix')
# plt.show()


Accuracy Generale del Modello: 0.9998

--- Report di Classificazione per Categoria ---
               precision    recall  f1-score   support

  Accessories       1.00      1.00      1.00      2255
      Apparel       1.00      1.00      1.00      4279
     Footwear       1.00      1.00      1.00      1844
   Free Items       1.00      1.00      1.00        21
Personal Care       1.00      1.00      1.00       481

     accuracy                           1.00      8880
    macro avg       1.00      1.00      1.00      8880
 weighted avg       1.00      1.00      1.00      8880



In [13]:
import joblib
joblib.dump(rf_model, 'category_classifier.joblib')
print("Modello salvato localmente.")

Modello salvato localmente.


In [14]:
from google.cloud import storage
import joblib 

# --- CONFIGURAZIONE ---
BUCKET_NAME = "data-science-project-luca-october-2025"  # Il tuo bucket GCS
MODEL_FILENAME = 'category_classifier.joblib'          # Il nome del file salvato
GCS_MODEL_PATH = "models/category_classifier.joblib"    # Il percorso dove lo salverai nel bucket
# ----------------------

def upload_model_to_gcs(bucket_name, source_file, destination_blob):
    """Carica l'artefatto (modello) sul bucket GCS."""
    print(f"Inizio il caricamento del modello {source_file} in GCS...")
    try:
        storage_client = storage.Client()
        bucket = storage_client.bucket(bucket_name)
        blob = bucket.blob(destination_blob)
        
        # Carica il modello
        blob.upload_from_filename(source_file)
        
        print(f"✅ SUCCESS: Modello caricato su gs://{bucket_name}/{destination_blob}")
    
    except Exception as e:
        print(f"ERRORE durante il caricamento GCS del modello: {e}")

# Esecuzione del caricamento
if __name__ == "__main__":
    upload_model_to_gcs(
        BUCKET_NAME, 
        MODEL_FILENAME, 
        GCS_MODEL_PATH
    )

Inizio il caricamento del modello category_classifier.joblib in GCS...
✅ SUCCESS: Modello caricato su gs://data-science-project-luca-october-2025/models/category_classifier.joblib


In [18]:
from google.cloud import bigquery

# --- CONFIGURAZIONE ---
DATASET_ID = "ecommerce_data"  
TABLE_ID = "product_features_for_ml" 
# ----------------------

# 1. Crea il client BigQuery
bq_client = bigquery.Client()

# 2. Definisci il riferimento alla Tabella
table_ref = bq_client.dataset(DATASET_ID).table(TABLE_ID)

# 3. Configurazione del job di caricamento (SOLO LE OPZIONI DI SCRITTURA)
# Non includiamo 'destination' qui!
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",  # Solo l'opzione di scrittura
)

# 4. Avvia il job di caricamento
# La destinazione (table_ref) è il SECONDO argomento del metodo
print(f"Inizio il caricamento del DataFrame in BigQuery: {DATASET_ID}.{TABLE_ID}")

job = bq_client.load_table_from_dataframe(
    df_save,           # 1° argomento: il DataFrame da caricare
    table_ref,         # 2° argomento: la destinazione (risolve l'errore!)
    job_config=job_config
)

job.result()  # Attendi la fine del job

print(f"✅ SUCCESS: Dati puliti e pronti per l'ML caricati in BigQuery.")
print(f"{job.output_rows} righe caricate.")

Inizio il caricamento del DataFrame in BigQuery: ecommerce_data.product_features_for_ml
✅ SUCCESS: Dati puliti e pronti per l'ML caricati in BigQuery.
44398 righe caricate.
